In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report
import copy

2025-07-17 14:31:06.750698: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-17 14:31:06.765436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752755466.787205   31618 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752755466.792555   31618 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752755466.808183   31618 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Load dataset

In [3]:
train_df = pd.read_csv('../data/train.txt', header=None, sep=";")
test_df = pd.read_csv('../data/test.txt', header=None, sep=";")
val_df = pd.read_csv('../data/validation.txt', header=None, sep=";")

In [4]:
train_df.head()

,0,1
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
train_df[1].value_counts()

1
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

We need to make a encoder for emotions

In [6]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df[1])
test_labels = label_encoder.transform(test_df[1])
val_labels = label_encoder.transform(val_df[1])

In [7]:
print(label_encoder.classes_)

['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_texts = train_df[0].tolist()
test_texts = test_df[0].tolist()
val_texts = val_df[0].tolist()

def get_embeddings(texts):
    batch_size = 32
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encodings = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        encodings = {k: v.to(device) for k, v in encodings.items()}

        with torch.no_grad():
            outputs = model(**encodings)
            cls_embeddings = outputs.last_hidden_state[:, 0]  # CLS token

        all_embeddings.append(cls_embeddings.cpu())

    embeds = torch.cat(all_embeddings)
    return embeds

X_train_tensor = get_embeddings(train_texts)
X_test_tensor = get_embeddings(test_texts)
X_val_tensor = get_embeddings(val_texts)

### Dataset & DataLoader

In [9]:
y_train_tensor = torch.tensor(train_labels, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels, dtype=torch.long)
y_val_tensor = torch.tensor(val_labels, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)

### Model

In [10]:
class FCNNModel(nn.Module):
    def __init__(self, input_size=768, num_classes=6):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        out = self.fc3(x)
        return out

In [11]:
def train(model, train_loader, optimizer, criterion, device, epochs=10, val_loader=None, patience=5):
    model.to(device)

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {total_loss:.4f} | Train Acc: {train_acc:.4f}", end="")

        if val_loader:
            val_loss, val_acc = evaluate(model, val_loader, criterion, device, return_metrics=True)
            print(f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

            # Early stopping logic
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                epochs_no_improve = 0
                best_model_wts = model.state_dict()
            else:
                epochs_no_improve += 1

            if epochs_no_improve >= patience:
                model.load_state_dict(best_model_wts)
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, "../models/best_fcnn_model.pth")
                break
        else:
            print()

In [12]:
def evaluate(model, data_loader, criterion, device, label_encoder=None, return_metrics=False):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = correct / total

    if return_metrics:
        avg_loss = total_loss / len(data_loader)
        return avg_loss, accuracy
    else:
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_ if label_encoder else None))

In [13]:
criterion = torch.nn.CrossEntropyLoss()
model = FCNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train

In [14]:
train(model, train_loader, optimizer, criterion, device, epochs=50, val_loader=test_loader, patience=5)

Epoch 1/50 | Train Loss: 685.7206 | Train Acc: 0.4763 | Val Loss: 1.2225 | Val Acc: 0.5445
Epoch 2/50 | Train Loss: 608.1442 | Train Acc: 0.5453 | Val Loss: 1.1279 | Val Acc: 0.5745
Epoch 3/50 | Train Loss: 582.1056 | Train Acc: 0.5656 | Val Loss: 1.1026 | Val Acc: 0.5815
Epoch 4/50 | Train Loss: 563.9680 | Train Acc: 0.5756 | Val Loss: 1.0664 | Val Acc: 0.5875
Epoch 5/50 | Train Loss: 550.1045 | Train Acc: 0.5854 | Val Loss: 1.0673 | Val Acc: 0.5985
Epoch 6/50 | Train Loss: 539.7444 | Train Acc: 0.5918 | Val Loss: 1.0519 | Val Acc: 0.6035
Epoch 7/50 | Train Loss: 531.3239 | Train Acc: 0.5984 | Val Loss: 1.0433 | Val Acc: 0.6000
Epoch 8/50 | Train Loss: 523.1310 | Train Acc: 0.6023 | Val Loss: 1.0242 | Val Acc: 0.6045
Epoch 9/50 | Train Loss: 516.2972 | Train Acc: 0.6115 | Val Loss: 1.0323 | Val Acc: 0.6030
Epoch 10/50 | Train Loss: 513.7974 | Train Acc: 0.6107 | Val Loss: 1.0425 | Val Acc: 0.6120
Epoch 11/50 | Train Loss: 506.5310 | Train Acc: 0.6176 | Val Loss: 1.0102 | Val Acc: 0.60

In [15]:
model = FCNNModel()
model.load_state_dict(torch.load("../models/best_fcnn_model.pth"))
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
evaluate(model, val_loader, criterion, label_encoder=label_encoder, device=device, return_metrics=False)


Classification Report:
              precision    recall  f1-score   support

       anger       0.54      0.32      0.40       275
        fear       0.57      0.48      0.52       212
         joy       0.79      0.69      0.73       704
        love       0.46      0.32      0.38       178
     sadness       0.52      0.84      0.64       550
    surprise       0.65      0.16      0.26        81

    accuracy                           0.60      2000
   macro avg       0.59      0.47      0.49      2000
weighted avg       0.62      0.60      0.59      2000

